In [190]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Чтение данных

In [191]:
data = pd.read_csv('train.csv')
data.sample(n=5)  # посмотрим на 5 случайных строк

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
1968,1,49.95,49.95,Female,0,No,No,Yes,Yes,DSL,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),1
2669,24,79.85,2001,Male,0,No,No,Yes,No,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Credit card (automatic),0
1721,13,83.20,1060.6,Male,0,No,No,Yes,No,Fiber optic,No,No,Yes,No,Yes,No,Month-to-month,Yes,Electronic check,1
878,1,19.55,19.55,Female,0,No,No,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,0
2551,71,111.25,7984.15,Female,0,No,No,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,0


In [192]:
# название колонок для числовых и категориальных признаков

# числовые признаки
numerical_columns = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# категориальные признаки
catigorial_columns = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

# все признаки
feature_columns = numerical_columns + catigorial_columns

# целевой признак
target_column = 'Churn'

In [193]:
data = data.replace('?', np.nan)
data = data.replace('', np.nan)
data = data.replace(' ', np.nan)

### Удалим все non'ы

Посмотрим есть ли незаполненные признаки (nan'ы)

In [194]:
data.isna().sum()

ClientPeriod                0
MonthlySpending             0
TotalSpent                  9
Sex                         0
IsSeniorCitizen             0
HasPartner                  0
HasChild                    0
HasPhoneService             0
HasMultiplePhoneNumbers     0
HasInternetService          0
HasOnlineSecurityService    0
HasOnlineBackup             0
HasDeviceProtection         0
HasTechSupportAccess        0
HasOnlineTV                 0
HasMovieSubscription        0
HasContractPhone            0
IsBillingPaperless          0
PaymentMethod               0
Churn                       0
dtype: int64

Видео, что есть nun в колонке TotalSpent. Эти строки с nan.

In [195]:
data = data.dropna()
data.isna().sum()

ClientPeriod                0
MonthlySpending             0
TotalSpent                  0
Sex                         0
IsSeniorCitizen             0
HasPartner                  0
HasChild                    0
HasPhoneService             0
HasMultiplePhoneNumbers     0
HasInternetService          0
HasOnlineSecurityService    0
HasOnlineBackup             0
HasDeviceProtection         0
HasTechSupportAccess        0
HasOnlineTV                 0
HasMovieSubscription        0
HasContractPhone            0
IsBillingPaperless          0
PaymentMethod               0
Churn                       0
dtype: int64

# Применение линейных моделей

### Предобработка данных

In [196]:
numeric_data = data[numerical_columns]
categorial_data = data[catigorial_columns]

In [197]:
# нормировка численных признаков

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_data = pd.DataFrame(scaler.fit_transform(numeric_data), columns=numerical_columns)
numeric_data.head()

,ClientPeriod,MonthlySpending,TotalSpent
0,0.919099,-1.506436,-0.557582
1,1.612060,-1.295997,-0.184763
2,-1.282072,0.362658,-0.976504
3,-0.018437,0.475334,0.122800
4,1.122911,1.666716,1.968909


In [198]:
# one-hot-encoding для категориальных признаков
categorial_data_dummy = pd.get_dummies(categorial_data)
categorial_data_dummy.head()

,IsSeniorCitizen,Sex_Female,Sex_Male,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,HasPhoneService_No,HasPhoneService_Yes,HasMultiplePhoneNumbers_No,...,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,False,True,False,True,False,True,False,True,True,...,False,False,True,False,True,False,False,False,False,True
1,0,False,True,False,True,True,False,False,True,False,...,False,False,False,True,True,False,False,True,False,False
2,0,False,True,True,False,True,False,False,True,True,...,False,True,False,False,False,True,False,False,True,False
3,1,True,False,False,True,True,False,False,True,False,...,False,True,False,False,True,False,False,False,False,True
4,0,True,False,False,True,False,True,False,True,False,...,True,False,False,True,True,False,False,True,False,False


In [208]:
datas = [numeric_data, categorial_data_dummy, data[target_column]]
processed_data = pd.concat(datas, axis=1).reindex(categorial_data_dummy.index)
processed_data = processed_data.dropna()
processed_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,IsSeniorCitizen,Sex_Female,Sex_Male,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,...,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,0.919099,-1.506436,-0.557582,0.0,False,True,False,True,False,True,...,False,True,False,True,False,False,False,False,True,0.0
1,1.612060,-1.295997,-0.184763,0.0,False,True,False,True,True,False,...,False,False,True,True,False,False,True,False,False,0.0
2,-1.282072,0.362658,-0.976504,0.0,False,True,True,False,True,False,...,True,False,False,False,True,False,False,True,False,1.0
3,-0.018437,0.475334,0.122800,1.0,True,False,False,True,True,False,...,True,False,False,True,False,False,False,False,True,0.0
4,1.122911,1.666716,1.968909,0.0,True,False,False,True,False,True,...,False,False,True,True,False,False,True,False,False,0.0


In [213]:
X = processed_data.drop(columns=target_column)
X

,ClientPeriod,MonthlySpending,TotalSpent,IsSeniorCitizen,Sex_Female,Sex_Male,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,...,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0.919099,-1.506436,-0.557582,0.0,False,True,False,True,False,True,...,False,False,True,False,True,False,False,False,False,True
1,1.612060,-1.295997,-0.184763,0.0,False,True,False,True,True,False,...,False,False,False,True,True,False,False,True,False,False
2,-1.282072,0.362658,-0.976504,0.0,False,True,True,False,True,False,...,False,True,False,False,False,True,False,False,True,False
3,-0.018437,0.475334,0.122800,1.0,True,False,False,True,True,False,...,False,True,False,False,True,False,False,False,False,True
4,1.122911,1.666716,1.968909,0.0,True,False,False,True,False,True,...,True,False,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5268,-1.200547,-1.145210,-0.973749,0.0,True,False,False,True,False,True,...,True,False,False,True,True,False,False,True,False,False
5269,0.715287,-0.679594,-0.045372,0.0,True,False,False,True,False,True,...,False,True,False,False,False,True,False,False,False,True
5270,-1.282072,-0.328310,-0.985693,0.0,False,True,True,False,True,False,...,True,True,False,False,False,True,False,False,True,False
5271,-0.140724,0.365972,-0.033560,0.0,False,True,True,False,True,False,...,True,False,True,False,False,True,True,False,False,False


In [214]:
y = processed_data[target_column]
y

0       0.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
5268    0.0
5269    0.0
5270    1.0
5271    0.0
5272    0.0
Name: Churn, Length: 5264, dtype: float64

### Заголовок

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC.

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C).


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

In [230]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [232]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

LabelEncoder()

# pipeline = make_pipeline(LabelEncoder())
# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test, y_test)

NameError: name 'a' is not defined

In [228]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

# Задание списка значений гиперпараметра C для тестирования
Cs = [100, 10, 1, 0.1, 0.01, 0.001]

# Инициализация и обучение модели с кросс-валидацией и подбором параметра C
model = LogisticRegressionCV(Cs=Cs, scoring='roc_auc', refit=True)
model.fit(X_train, y_train)

# Получение лучшего значения параметра C
best_C = model.C_[0]
print("Лучшее значение параметра C:", best_C)

# Предсказание на валидационной выборке
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Вычисление ROC-AUC метрики на валидационной выборке
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC на валидационной выборке:", roc_auc)


Лучшее значение параметра C: 0.1
ROC-AUC на валидационной выборке: 0.8284780667192013


In [229]:
X_test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission.csv')

y_pred_proba = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC на валидационной выборке:", roc_auc)
# submission['Churn'] = # используйте best_model.predict_proba(X_test), не забудьте выделить вероятность класса 1.
# submission.to_csv('./my_submission.csv')

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- HasChild
- HasContractPhone
- HasDeviceProtection
- HasInternetService
- HasMovieSubscription
- ...
Feature names seen at fit time, yet now missing:
- HasChild_No
- HasChild_Yes
- HasContractPhone_Month-to-month
- HasContractPhone_One year
- HasContractPhone_Two year
- ...


In [233]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Название категориальных колонок
categorical_columns = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild'
]

# Создание пайплайна
pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('categorical', LabelEncoder(), categorical_columns)
        ],
        remainder='passthrough'
    ),
    SimpleImputer(strategy='mean'),  # Можно изменить стратегию в зависимости от вашего набора данных
    LogisticRegression()
)

# Обучение пайплайна на данных
# pipeline.fit(X, y)

# Предсказание с использованием пайплайна
# y_pred = pipeline.predict(X)


In [234]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Генерация синтетических данных
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Создание DataFrame для данных
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y

# Разделение на train/test выборку
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Создание и обучение пайплайна
pipeline.fit(X_train, y_train)

# Предсказание с использованием пайплайна
y_pred = pipeline.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


ValueError: A given column is not a column of the dataframe